In [1]:
print("hi")

hi


In [8]:
import requests
import pandas as pd
from datetime import datetime
import json

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        # If we found target records, add them
        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Check if we've moved past our target date (since we're sorting DESC)
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records

In [ ]:
if __name__ == "__main__":
    target_date = '2025-10-31'

    print(f"Searching for BOAMP records with publication date: {target_date}")
    print("=" * 60)

    # Get all records
    all_records = get_all_records_for_date(target_date)

    print(f"\n{'='*60}")
    print(f"EXTRACTION COMPLETE")
    print(f"Found {len(all_records)} records for date {target_date}")

Searching for BOAMP records with publication date: 2025-10-31
Requesting offset 0...
Retrieved 100 records for 2025-10-31... Total so far: 100
Requesting offset 100...
Retrieved 100 records for 2025-10-31... Total so far: 200
Requesting offset 200...
Retrieved 100 records for 2025-10-31... Total so far: 300
Requesting offset 300...
Retrieved 100 records for 2025-10-31... Total so far: 400
Requesting offset 400...
Retrieved 59 records for 2025-10-31... Total so far: 459
Reached dates earlier than 2025-10-31. Stopping.

EXTRACTION COMPLETE
Found 459 records for date 2025-10-31
Searching for BOAMP records with publication date: 2025-10-30
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Requesting offset 300...
Requesting offset 400...
Retrieved 41 records for 2025-10-30... Total so far: 41
Requesting offset 500...
Retrieved 100 records for 2025-10-30... Total so far: 141
Requesting offset 600...
Retrieved 100 records for 2025-10-30... Total so far: 241
Requesting 

In [24]:
def create_excel_simple(records, target_date):
    """Simple and robust Excel creation"""

    # Clean the records
    cleaned_records = []
    for record in records:
        cleaned_record = {}
        for key, value in record.items():
            # Handle different data types
            if isinstance(value, (list, dict)):
                cleaned_record[key] = json.dumps(value, ensure_ascii=False)
            elif value is None:
                cleaned_record[key] = ''
            else:
                cleaned_record[key] = value
        cleaned_records.append(cleaned_record)

    # Create DataFrame
    df = pd.DataFrame(cleaned_records)

    # Create filename
    filename = f"BOAMP_{target_date}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

    # Export to Excel - SIMPLE VERSION, just the data
    df.to_excel(filename, index=False, engine='openpyxl')

    return filename, df

In [25]:
if all_records:
    # Create Excel file
    excel_filename, df = create_excel_simple(all_records, target_date)

    print(f"\n✅ Excel file created: {excel_filename}")
    print(f"📊 Total records exported: {len(all_records)}")
    print(f"📋 Total columns: {len(df.columns)}")

    # Show column names
    print(f"\n📋 Columns in Excel file:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")

    # Display sample data
    print(f"\n📄 SAMPLE DATA (first 3 records):")
    print(f"{'='*60}")
    for i, record in enumerate(all_records[:3], 1):
        print(f"Record {i}:")
        print(f"  Title: {record.get('objet', 'N/A')[:80]}...")
        print(f"  Buyer: {record.get('nomacheteur', 'N/A')}")
        print(f"  Procedure: {record.get('procedure_libelle', 'N/A')}")
        print(f"  Date: {record.get('dateparution', 'N/A')}")
        print("-" * 60)
else:
    print("❌ No records found for the specified date.")


✅ Excel file created: BOAMP_2025-10-29_20251101_003445.xlsx
📊 Total records exported: 463
📋 Total columns: 41

📋 Columns in Excel file:
   1. idweb
   2. id
   3. contractfolderid
   4. objet
   5. filename
   6. famille
   7. code_departement
   8. code_departement_prestation
   9. famille_libelle
  10. dateparution
  11. datefindiffusion
  12. datelimitereponse
  13. nomacheteur
  14. titulaire
  15. perimetre
  16. type_procedure
  17. soustype_procedure
  18. procedure_libelle
  19. procedure_categorise
  20. nature
  21. sousnature
  22. nature_libelle
  23. sousnature_libelle
  24. nature_categorise
  25. nature_categorise_libelle
  26. criteres
  27. marche_public_simplifie
  28. marche_public_simplifie_label
  29. etat
  30. descripteur_code
  31. dc
  32. descripteur_libelle
  33. type_marche
  34. type_marche_facette
  35. type_avis
  36. annonce_lie
  37. annonces_anterieures
  38. source_schema
  39. gestion
  40. donnees
  41. url_avis

📄 SAMPLE DATA (first 3 records):
Re

In [15]:
import pandas as pd

# --- 1. Define your file path and keyword ---
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_20251031_235226.xlsx"
keyword = "serrurerie"
output_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx"

# --- 2. Load the Excel file ---
df = pd.read_excel(file_path)

# --- 3. Convert all text to lowercase for case-insensitive search ---
df_str = df.astype(str).apply(lambda x: x.str.lower())

# --- 4. Filter rows that contain the keyword in any column ---
mask = df_str.apply(lambda x: x.str.contains(keyword, na=False))
filtered_df = df[mask.any(axis=1)]

# --- 5. Save the filtered data to a new Excel file ---
filtered_df.to_excel(output_path, index=False)

print(f"✅ Found {len(filtered_df)} matching rows. Saved to:\n{output_path}")


✅ Found 15 matching rows. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx


In [18]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# --- 1. Paths & Keyword ---
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx"
output_path = r"D:\sorabo\sorabo\BOAMP.xlsx"
keyword = "serrurerie"

# --- 2. Load Excel file into pandas ---
df = pd.read_excel(file_path)

# --- 3. Save temporarily (openpyxl works on real Excel files) ---
df.to_excel(output_path, index=False)

# --- 4. Load workbook with openpyxl ---
wb = load_workbook(output_path)
ws = wb.active

# --- 5. Define the green fill style ---
green_fill = PatternFill(start_color="90EE90", end_color="90EE90", fill_type="solid")  # Light green

# --- 6. Loop through each cell and check for keyword ---
for row in ws.iter_rows(min_row=2):  # skip header row
    for cell in row:
        if cell.value and keyword.lower() in str(cell.value).lower():
            cell.fill = green_fill

# --- 7. Save workbook ---
wb.save(output_path)

print(f"✅ Highlighted all cells containing '{keyword}' in green.\nSaved to:\n{output_path}")


✅ Highlighted all cells containing 'serrurerie' in green.
Saved to:
D:\sorabo\sorabo\BOAMP.xlsx


In [19]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# --- File paths ---
input_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx"
output_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie_highlighted.xlsx"

# --- Keyword to highlight ---
keyword = "serrurerie"

# --- Load Excel file with openpyxl directly ---
wb = load_workbook(input_path)
ws = wb.active

# --- Define the green fill ---
green_fill = PatternFill(start_color="90EE90", end_color="90EE90", fill_type="solid")

# --- Loop through cells and apply highlight ---
count = 0
for row in ws.iter_rows(min_row=1):  # Include headers if needed
    for cell in row:
        if cell.value and keyword.lower() in str(cell.value).lower():
            cell.fill = green_fill
            count += 1

# --- Save the new file ---
wb.save(output_path)

print(f"✅ Highlighted {count} cells containing '{keyword}'.")
print(f"💾 Saved to: {output_path}")


✅ Highlighted 17 cells containing 'serrurerie'.
💾 Saved to: D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie_highlighted.xlsx


In [20]:
import pandas as pd
import os

# --- 1. Define paths and keywords ---
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_20251031_235226.xlsx"
output_dir = r"D:\sorabo\sorabo"
keywords = [
    "serrurerie",
    "miroiterie",
    "métallerie",
    "menuiserie extérieure",
    "clôtures",
    "44233000",
    "escaliers",
    "44316500",
    "45421132",
    "45421140"
]

# --- 2. Load the Excel file ---
df = pd.read_excel(file_path)
df_str = df.astype(str).apply(lambda x: x.str.lower())  # lowercase for case-insensitive match

# --- 3. Prepare a list to store all matches ---
all_matches = pd.DataFrame()

# --- 4. Loop through each keyword ---
for keyword in keywords:
    mask = df_str.apply(lambda x: x.str.contains(keyword.lower(), na=False))
    filtered_df = df[mask.any(axis=1)]
    
    if not filtered_df.empty:
        output_path = os.path.join(output_dir, f"BOAMP_2025-10-31_{keyword.replace(' ', '_')}.xlsx")
        filtered_df.to_excel(output_path, index=False)
        print(f"✅ Found {len(filtered_df)} rows for '{keyword}'. Saved to:\n{output_path}")
        
        # Add a column showing which keyword matched
        filtered_df = filtered_df.copy()
        filtered_df["keyword"] = keyword
        all_matches = pd.concat([all_matches, filtered_df], ignore_index=True)
    else:
        print(f"⚠️ No matches found for '{keyword}'.")

# --- 5. Save all combined matches ---
if not all_matches.empty:
    combined_output = os.path.join(output_dir, "BOAMP_2025-10-31_ALL_KEYWORDS.xlsx")
    all_matches.to_excel(combined_output, index=False)
    print(f"\n📊 Combined results saved to:\n{combined_output}")
else:
    print("\n❌ No matches found for any keyword.")


✅ Found 15 rows for 'serrurerie'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx
⚠️ No matches found for 'miroiterie'.
✅ Found 7 rows for 'métallerie'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_métallerie.xlsx
✅ Found 4 rows for 'menuiserie extérieure'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_menuiserie_extérieure.xlsx
✅ Found 3 rows for 'clôtures'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_clôtures.xlsx
⚠️ No matches found for '44233000'.
✅ Found 1 rows for 'escaliers'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_escaliers.xlsx
✅ Found 4 rows for '44316500'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_44316500.xlsx
⚠️ No matches found for '45421132'.
✅ Found 2 rows for '45421140'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_45421140.xlsx

📊 Combined results saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDS.xlsx


In [29]:
import pandas as pd
import os

# --- 1. Define your file path ---
file_path = r"D:\sorabo\sorabo\boamp_records_20251101_004355.xlsx"
output_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDS.xlsx"

# --- 2. Define all your keywords and CPV codes ---
keywords = [
    # Secteurs d’activité
    "serrurerie",
    "miroiterie",
    "métallerie",
    "menuiserie extérieure",
    "clôtures",
    "escaliers",

    # CPV simples
    "44233000",
    "44316500",
    "45421132",
    "45421140",

    # CPV détaillés avec description
    "45420000-7", "Travaux de menuiserie et de charpenterie",
    "45421000-4", "Travaux de menuiserie",
    "45421100-5", "Pose de portes et de fenêtres et d'éléments accessoires",
    "45421110-8", "Pose d'encadrements de portes et de fenêtres",
    "45421111-5", "Pose d'encadrements de portes",
    "45421112-2", "Pose d'encadrements de fenêtres",
    "45421120-1", "Pose de seuils",
    "45421130-4", "Poses de portes et de fenêtres",
    "45421131-1", "Pose de portes",
    "45421132-8", "Pose de fenêtres",
    "45421140-7", "Pose de menuiseries métalliques, excepté portes et fenêtres",
    "45421141-4", "Travaux de cloisonnement",
    "45421142-1", "Installation de volets",
    "45421143-8", "Travaux d'installation de stores",
    "45421144-5", "Travaux d'installation de vélums",
    "45421145-2", "Travaux d'installation de volets roulants"
]

# --- 3. Load Excel ---
print("📂 Reading Excel file...")
df = pd.read_excel(file_path)
df_str = df.astype(str).apply(lambda x: x.str.lower())  # for case-insensitive search

# --- 4. Create one big dataframe for all results ---
all_matches = pd.DataFrame()

# --- 5. Loop through each keyword ---
for keyword in keywords:
    mask = df_str.apply(lambda x: x.str.contains(keyword.lower(), na=False))
    filtered_df = df[mask.any(axis=1)]
    
    if not filtered_df.empty:
        print(f"✅ Found {len(filtered_df)} rows for '{keyword}'")
        filtered_df = filtered_df.copy()
        filtered_df["keyword"] = keyword
        all_matches = pd.concat([all_matches, filtered_df], ignore_index=True)
    else:
        print(f"⚠️ No matches found for '{keyword}'.")

# --- 6. Save everything to one file ---
if not all_matches.empty:
    all_matches.to_excel(output_path, index=False)
    print(f"\n📊 All matching data saved successfully to:\n{output_path}")
else:
    print("\n❌ No matches found for any keyword.")


📂 Reading Excel file...
✅ Found 69 rows for 'serrurerie'
✅ Found 4 rows for 'miroiterie'
✅ Found 20 rows for 'métallerie'
✅ Found 13 rows for 'menuiserie extérieure'
✅ Found 14 rows for 'clôtures'
✅ Found 6 rows for 'escaliers'
⚠️ No matches found for '44233000'.
✅ Found 21 rows for '44316500'
⚠️ No matches found for '45421132'.
✅ Found 5 rows for '45421140'
⚠️ No matches found for '45420000-7'.
⚠️ No matches found for 'Travaux de menuiserie et de charpenterie'.
⚠️ No matches found for '45421000-4'.
✅ Found 5 rows for 'Travaux de menuiserie'
⚠️ No matches found for '45421100-5'.
⚠️ No matches found for 'Pose de portes et de fenêtres et d'éléments accessoires'.
⚠️ No matches found for '45421110-8'.
⚠️ No matches found for 'Pose d'encadrements de portes et de fenêtres'.
⚠️ No matches found for '45421111-5'.
⚠️ No matches found for 'Pose d'encadrements de portes'.
⚠️ No matches found for '45421112-2'.
⚠️ No matches found for 'Pose d'encadrements de fenêtres'.
⚠️ No matches found for '4542

In [27]:
import requests
import pandas as pd
from datetime import datetime
import json

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        # If we found target records, add them
        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Check if we've moved past our target date (since we're sorting DESC)
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records


if __name__ == "__main__":
    # Multiple dates
    target_dates = ['2025-10-31', '2025-10-30', '2025-10-29', '2025-10-28', '2025-10-27']
    all_results = []

    for target_date in target_dates:
        print(f"\n{'='*60}")
        print(f"Searching for BOAMP records with publication date: {target_date}")
        print("=" * 60)

        records = get_all_records_for_date(target_date)
        print(f"Found {len(records)} records for date {target_date}")
        all_results.extend(records)

    # Save all data in one JSON file
    output_file = f"boamp_records_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    print(f"\n{'='*60}")
    print(f"✅ EXTRACTION COMPLETE — Total {len(all_results)} records from {len(target_dates)} dates.")
    print(f"💾 Data saved to {output_file}")



Searching for BOAMP records with publication date: 2025-10-31
Requesting offset 0...
Retrieved 100 records for 2025-10-31... Total so far: 100
Requesting offset 100...
Retrieved 100 records for 2025-10-31... Total so far: 200
Requesting offset 200...
Retrieved 100 records for 2025-10-31... Total so far: 300
Requesting offset 300...
Retrieved 100 records for 2025-10-31... Total so far: 400
Requesting offset 400...
Retrieved 59 records for 2025-10-31... Total so far: 459
Reached dates earlier than 2025-10-31. Stopping.
Found 459 records for date 2025-10-31

Searching for BOAMP records with publication date: 2025-10-30
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Requesting offset 300...
Requesting offset 400...
Retrieved 41 records for 2025-10-30... Total so far: 41
Requesting offset 500...
Retrieved 100 records for 2025-10-30... Total so far: 141
Requesting offset 600...
Retrieved 100 records for 2025-10-30... Total so far: 241
Requesting offset 700...
Retri

In [28]:
import requests
import pandas as pd
from datetime import datetime

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Stop if we’ve reached dates earlier than target_date
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records


if __name__ == "__main__":
    target_dates = ['2025-10-31', '2025-10-30', '2025-10-29', '2025-10-28', '2025-10-27']
    all_results = []

    for target_date in target_dates:
        print(f"\n{'='*60}")
        print(f"Fetching BOAMP records for: {target_date}")
        records = get_all_records_for_date(target_date)
        print(f"→ Found {len(records)} records for {target_date}")
        all_results.extend(records)

    print(f"\nTotal records retrieved: {len(all_results)}")

    # ✅ Save results to Excel
    if all_results:
        df = pd.DataFrame(all_results)
        output_file = f"boamp_records_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"💾 Data saved to Excel file: {output_file}")
    else:
        print("⚠️ No records found for the given dates.")



Fetching BOAMP records for: 2025-10-31
Requesting offset 0...
Retrieved 100 records for 2025-10-31... Total so far: 100
Requesting offset 100...
Retrieved 100 records for 2025-10-31... Total so far: 200
Requesting offset 200...
Retrieved 100 records for 2025-10-31... Total so far: 300
Requesting offset 300...
Retrieved 100 records for 2025-10-31... Total so far: 400
Requesting offset 400...
Retrieved 59 records for 2025-10-31... Total so far: 459
Reached dates earlier than 2025-10-31. Stopping.
→ Found 459 records for 2025-10-31

Fetching BOAMP records for: 2025-10-30
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Requesting offset 300...
Requesting offset 400...
Retrieved 41 records for 2025-10-30... Total so far: 41
Requesting offset 500...
Retrieved 100 records for 2025-10-30... Total so far: 141
Requesting offset 600...
Retrieved 100 records for 2025-10-30... Total so far: 241
Requesting offset 700...
Retrieved 100 records for 2025-10-30... Total so far: 

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_all_records_for_date(target_date, max_records=100000):
    """Get all records for a specific date from BOAMP"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"❌ Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break

        # Filter for our date
        target_records = [r for r in records if r.get('dateparution') == target_date]

        if target_records:
            all_records.extend(target_records)
            print(f"✅ {len(target_records)} records found for {target_date} (total {len(all_records)})")

        # Stop once we passed earlier dates
        if records and records[-1].get('dateparution', '') < target_date:
            break

        offset += limit
        if offset > 10000:
            print("⚠️ Safety limit reached, stopping.")
            break

    return all_records


if __name__ == "__main__":
    start_date = datetime(2025, 1, 1)
    end_date = datetime(2025, 10, 31)
    current_date = start_date
    all_results = []

    print(f"\n🔎 Extracting BOAMP records from {start_date.date()} to {end_date.date()}")
    print("=" * 60)

    while current_date <= end_date:
        target_date = current_date.strftime("%Y-%m-%d")
        print(f"\n📅 Processing {target_date}...")
        records = get_all_records_for_date(target_date)
        if records:
            all_results.extend(records)
        current_date += timedelta(days=1)

    print(f"\n{'='*60}")
    print(f"✅ EXTRACTION COMPLETE — Total records: {len(all_results)}")

    # Save all results to Excel
    if all_results:
        df = pd.DataFrame(all_results)
        output_file = f"boamp_2025_01_01_to_2025_10_31_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"💾 Data saved to Excel file: {output_file}")
    else:
        print("⚠️ No records found for the selected period.")



🔎 Extracting BOAMP records from 2025-01-01 to 2025-10-31

📅 Processing 2025-01-01...
❌ Error 400: {
  "error_code": "InvalidRESTParameterError",
  "message": "Invalid value for sum of offset + limit API parameter: 10100 was found but <= 10000 is expected."
}

📅 Processing 2025-01-02...
❌ Error 400: {
  "error_code": "InvalidRESTParameterError",
  "message": "Invalid value for sum of offset + limit API parameter: 10100 was found but <= 10000 is expected."
}

📅 Processing 2025-01-03...


In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"❌ Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter only those with our date
        target_records = [r for r in records if r.get('dateparution') == target_date]

        if target_records:
            all_records.extend(target_records)
            print(f"✅ {len(target_records)} records found for {target_date} (total {len(all_records)})")

        # Stop if we’ve reached earlier dates (since sorted DESC)
        if records and records[-1].get('dateparution', '') < target_date:
            break

        offset += limit

        if offset > 10000:
            print("⚠️ Safety limit reached. Stopping.")
            break

    return all_records


if __name__ == "__main__":
    # 🔹 Define the date range for October 2025
    start_date = datetime(2025, 10, 1)
    end_date = datetime(2025, 10, 31)
    current_date = start_date
    all_results = []

    print(f"\n🔎 Extracting BOAMP records from {start_date.date()} to {end_date.date()}")
    print("=" * 60)

    while current_date <= end_date:
        target_date = current_date.strftime("%Y-%m-%d")
        print(f"\n📅 Processing {target_date}...")
        records = get_all_records_for_date(target_date)
        if records:
            all_results.extend(records)
        current_date += timedelta(days=1)

    print(f"\n{'='*60}")
    print(f"✅ EXTRACTION COMPLETE — Total records: {len(all_results)}")

    # 🔹 Save to Excel
    if all_results:
        df = pd.DataFrame(all_results)
        output_file = f"boamp_octobre_2025_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"💾 Data saved to Excel file: {output_file}")
    else:
        print("⚠️ No records found for the selected period.")



🔎 Extracting BOAMP records from 2025-10-01 to 2025-10-31

📅 Processing 2025-10-01...


KeyboardInterrupt: 

In [13]:
import requests
import pandas as pd
from datetime import datetime

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Stop if we’ve reached dates earlier than target_date
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records


if __name__ == "__main__":
    target_dates = ['2025-10-19', '2025-10-18', '2025-10-17', '2025-10-16', '2025-10-15', '2025-10-14', '2025-10-13', '2025-10-12', '2025-10-11', '2025-10-10']
    all_results = []

    for target_date in target_dates:
        print(f"\n{'='*60}")
        print(f"Fetching BOAMP records for: {target_date}")
        records = get_all_records_for_date(target_date)
        print(f"→ Found {len(records)} records for {target_date}")
        all_results.extend(records)

    print(f"\nTotal records retrieved: {len(all_results)}")

    # ✅ Save results to Excel
    if all_results:
        df = pd.DataFrame(all_results)
        output_file = f"boamp_records_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"💾 Data saved to Excel file: {output_file}")
    else:
        print("⚠️ No records found for the given dates.")



Fetching BOAMP records for: 2025-10-19
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Requesting offset 300...
Requesting offset 400...
Requesting offset 500...
Requesting offset 600...
Requesting offset 700...
Requesting offset 800...
Requesting offset 900...
Requesting offset 1000...
Requesting offset 1100...
Requesting offset 1200...
Requesting offset 1300...
Requesting offset 1400...
Requesting offset 1500...
Requesting offset 1600...
Requesting offset 1700...
Requesting offset 1800...
Requesting offset 1900...
Requesting offset 2000...
Requesting offset 2100...
Requesting offset 2200...
Requesting offset 2300...
Requesting offset 2400...
Requesting offset 2500...
Requesting offset 2600...
Requesting offset 2700...
Requesting offset 2800...
Requesting offset 2900...
Requesting offset 3000...
Requesting offset 3100...
Requesting offset 3200...
Requesting offset 3300...
Requesting offset 3400...
Requesting offset 3500...
Requesting offset 3600...
Requesting

In [12]:
from datetime import datetime, timedelta

dates_octobre = [
    (datetime(2025, 10, 19) - timedelta(days=i)).strftime("%Y-%m-%d")
    for i in range(10)
]

print(dates_octobre)


['2025-10-19', '2025-10-18', '2025-10-17', '2025-10-16', '2025-10-15', '2025-10-14', '2025-10-13', '2025-10-12', '2025-10-11', '2025-10-10']
